In [179]:
import pandas as pd
import numpy as np
import re

In [180]:
# loading in data
df = pd.read_parquet('final_df.parquet')

# inspecting data
df.head()

,Size,Open Viewing,Lot,Bathrooms,Time,Postcode,Heating,Status,Bedrooms,Rates,Receptions,Broadband,Price (£),Venue,Postcode_number,Address,Style,Estate Agent,Belfast District
0,None,None,None,2,None,BT8 8FX,None,None,4,1652.05,2,900Mbps,335000.0,None,BT8,5 Haddo Crescent,Townhouse,Ulster Property Sales (Forestside),South
1,None,None,None,1,None,BT7 3DQ,Gas,None,1,826.03,1,900Mbps,129950.0,None,BT7,5 Annadale Green,Apartment,Nicholas Residential,South
2,None,None,None,2,None,BT8 6GY,Gas,For sale,4,1296.73,1,900Mbps,239950.0,None,BT8,11 Finsbury Drive,Semi-detached House,James Duggan Estates,South
3,None,None,None,None,None,BT7 3JH,Gas,None,3,1391.20,1,80Mbps,219950.0,None,BT7,"Apt 13 Anna Hill, 2 Annadale Avenue",Apartment,Templeton Robinson (Lisburn Road),South
4,None,None,None,4,None,BT8 8LN,None,None,6,None,3,None,675000.0,None,BT8,103 Fort Road,Detached House,Fetherstons (South Belfast Office),South


# After initial inspection we can see that there are several columns that will not add anything to our analysis so we will get rid of them. Additionally, there are columns where the values will need to be cleaned and null values addressed.

In [181]:
# Dropping columns we don't need
df = df.drop(columns=['Size', 'Open Viewing', 'Lot', 'Time', 'Status', 'Venue', 'Heating', 'Broadband'])

# Removing the leading white space from the Postcode column
df['Postcode'] = df['Postcode'].apply(lambda x: x.lstrip() if pd.notna(x) else x)

# It appears we have pulled in 'Belfast' in the postcode column, which clearly shouldn't be there. Rather than discarding these values I can see that the vast majority are in 'The Hamptons' and the 'Orchard Hall Apartments' thus I should be able to find a postcode for these properties and update the dataset.

In [182]:
# Updating postcode numbers

df['Postcode_number'] = np.where(df['Address'].str.contains('The Hamptons'), 'BT7', df['Postcode_number'])
df['Postcode_number'] = np.where(df['Address'].str.contains('Orchard Hall'), 'BT4', df['Postcode_number'])
df['Postcode_number'] = np.where(df['Address'].str.contains('The Admin Building'), 'BT8', df['Postcode_number'])
df['Postcode_number'] = np.where(df['Address'].str.contains('Lesley Park Demesne'), 'BT9', df['Postcode_number'])

In [183]:
# Now we will check to see how many postcodes say 'Belfast'
df.loc[df['Postcode_number'] == 'Belfast']

,Bathrooms,Postcode,Bedrooms,Rates,Receptions,Price (£),Postcode_number,Address,Style,Estate Agent,Belfast District
16,3,Belfast,5,None,2,499950.0,Belfast,"The Bracken, Ferndene",Detached House,Simon Brien Residential (East Belfast),None
585,None,Belfast,4,None,1,NaN,Belfast,"The Willow, House Type S, Castlehill Wood",Semi-detached House,Simon Brien Residential (East Belfast),None
603,2,Belfast,4,None,2,695000.0,Belfast,"Phase 2, Madigan Hill",Detached House,Marketed by multiple agents,None
617,2,Belfast City Centre,2,None,1,225000.0,Belfast,"Two Bedrooms, Coyles Place At Portland 88",Apartments,Colliers New Homes,None
889,3,Belfast,4,None,2,565000.0,Belfast,"The Pottinger, Baron Lane",Detached House,Colliers New Homes,None
932,1,Belfast City Centre,1,None,1,165000.0,Belfast,"One Bedroom, Coyles Place At Portland 88",Apartment,Colliers New Homes,None
1023,2,Belfast,2,None,1,NaN,Belfast,"2 Bedroom Apartment, 100 Kings Road",Apartment,Marketed by multiple agents,None


# It would be possible to get a postcode for the remaining 7 properties by searching each individual address, instead, in the interest of time we will remove these properties from the dataset.

In [184]:
df['Estate Agent'].unique()

array(['Ulster Property Sales (Forestside)', 'Nicholas Residential',
       'James Duggan Estates', 'Templeton Robinson (Lisburn Road)',
       'Fetherstons (South Belfast Office)',
       'Fetherstons (East Belfast Office)',
       'Ulster Property Sales (Andersonstown)', 'CPS (Belfast)',
       'Reeds Rains (Glengormley)', 'NorthernProperty.com (Residential)',
       'McClearys Property Sales', 'Andrews & Gregg (Dundonald)',
       'Simon Brien Residential (East Belfast)',
       'Ulster Property Sales (Cavehill)', 'Dougan Residential',
       'Property People', 'Macfarlane & Smyth',
       'John Minnis Estate Agents',
       'Michael Chandler Estate Agents (Belfast)', 'Pinpoint Property',
       'Privilege Properties', 'GOC Estate Agents',
       'Forsythe Residential', 'M&H Property Rentals & Management',
       'TMC Estate Agents (Andersonstown)',
       'Cairns & Downing Sales and Lettings', 'BRG Gibson Auctions',
       'Natalie Clarke Residential', 'Rodgers Residential',
      

# It would be interesting to compare how each estate agent is performing amongst one another so I will remove the office in parentheses e.g (Andersontown).

In [185]:
# Function to remove parentheses from estate agent
def remove_parentheses(row):
    return re.sub(r'\([^)]*\)', '', row)

df['Estate Agent'] = df['Estate Agent'].apply(lambda x: remove_parentheses(x) if pd.notna(x) else x)

In [186]:
# Inspecting different property styles
df['Style'].unique()

array(['Townhouse', 'Apartment', 'Semi-detached House', 'Detached House',
       'Terrace House', 'Semi-detached Bungalow', 'End Townhouse',
       'Mid-terrace House', '3rd Floor Apartment', 'Detached Villa',
       'End-terrace House', 'House', 'Detached Bungalow',
       'Ground Floor Apartment', 'Mid Townhouse', 'Apartments',
       'Extended Mid-terrace', 'Semi-detached Villa', 'Duplex Apartment',
       'Penthouse Apartment', '1st Floor Apartment', 'Ground Floor Flat',
       'Detached Chalet', 'Apartment / Flat', 'Bungalow',
       '2nd Floor Apartment', 'Semi-detached Chalet',
       'Link-detached house', 'Detached Duplex Bungalow',
       'Detached with annex', 'Detached Chalet Bungalow',
       'House and Land', 'Semi-detached Chalet Bungalow', 'Duplex Flat',
       'Flat'], dtype=object)

# After inspection, we can see that there are 35 different property styles in the dataset, this is far too many. For the sake of our analysis we would like to get this number down as a low as possible. To do this we will use the replace method available on strings to search for certain keywords, for example, the first line of code that follows searches for anything that starts or ends with the words 'flat' or 'apartment' and converts the value to just say 'Apartment'. This allows each style to be correctly grouped together within one style rather than existing across multiple styles for the sake of a few words. A similar approach is taken to other styles including: Detached, Semi-detached, Townhouse, Bungalow and Terrace.

In [187]:
# Cleaning up styles
df['Style'] = df['Style'].str.replace(r'.*(flat|apartment).*', 'Apartment', case=False, regex=True)
df['Style'] = df['Style'].str.replace(r'.*(link|semi-detached).*', 'Semi-detached House', case=False, regex=True)
df['Style'] = df['Style'].str.replace(r'.*(terrace).*', 'Terrace House', case=False, regex=True)
df['Style'] = df['Style'].str.replace(r'.*(townhouse).*', 'Townhouse', case=False, regex=True)
df['Style'] = df['Style'].str.replace(r'.*(bungalow).*', 'Bungalow', case=False, regex=True)
df['Style'] = df['Style'].apply(lambda x: x.replace(x, 'Detached House') if x in ['Detached Villa', 'Detached Chalet', 'Detached with annex'] else x)

# Finally because 'House' and 'House and Land' don't add to our analysis we will remove them from the dataset.
styles_to_remove = ['House','House and Land']
df = df[~df['Style'].isin(styles_to_remove)]

In [188]:
df['Style'].unique()

array(['Townhouse', 'Apartment', 'Semi-detached House', 'Detached House',
       'Terrace House', 'Bungalow'], dtype=object)

# We now have a manageable number of property styles that we will use in our subsequent analysis.

# The next stage is to investigate the Price column to see if we have any outliers that need addressed. Upon inspection there are a few outliers contained with this data, however, given the nature of this analysis I have chosen to only use a 10% - 90% quantile range as it is natural to expect higher and lower property values within certain postcode areas. Important to note that each quantile range is a reflection of the postcode number and style of property.

In [189]:
# Remove Outliers
# Calculate the IQR for each unique postcode
iqr_per_postcode = pd.DataFrame(df.groupby(['Postcode_number','Style'])['Price (£)'].quantile([0.1, 0.9]).unstack()).reset_index()
iqr_per_postcode.columns = ['Postcode_number','Style', 'Q1', 'Q3']
iqr_per_postcode

,Postcode_number,Style,Q1,Q3
0,BT1,Apartment,122500.0,407475.0
1,BT10,Apartment,132980.0,204980.0
2,BT10,Bungalow,325000.0,325000.0
3,BT10,Detached House,260985.0,324985.0
4,BT10,Semi-detached House,180980.0,290980.0
...,...,...,...,...
79,BT9,Townhouse,168000.0,557000.0
80,Belfast,Apartment,171000.0,219000.0
81,Belfast,Detached House,512960.0,669000.0
82,Belfast,Semi-detached House,NaN,NaN


In [190]:
# Merge the IQR values back into the original DataFrame
df = df.merge(iqr_per_postcode, on=['Postcode_number','Style'])
# Filter the rows based on the IQR values
df = df[(df['Price (£)'] >= df['Q1']) & (df['Price (£)'] <= df['Q3'])]

# Drop the 10% and 90% quantile columns
df.drop(columns=['Q1','Q3'], inplace=True)

# The next stage is to deal with the null Rates values. I have decided to use the average rates for each property style within a postcode number. This ensures, for example, an apartment within BT4 gets a rates value that reflects another apartment in BT4 rather than a rates value that is distorted by a detached home in BT9.

In [191]:
# Updating null rates values
df['Rates'] = pd.to_numeric(df['Rates'], errors='coerce')
avg_rates_by_group = df.groupby(['Postcode_number','Style'])['Rates'].mean().round(0)

# Merge the average rates with the original DataFrame
df = df.merge(avg_rates_by_group, on=['Postcode_number', 'Style'], how='left', suffixes=('', '_avg'))

# Fill null values with the corresponding average rate
df['Rates'].fillna(df['Rates_avg'], inplace=True)
df.drop(columns=['Rates_avg'], inplace=True)


In [192]:
avg_rates_by_group

Postcode_number  Style              
BT1              Apartment              1174.0
BT10             Apartment              1000.0
                 Bungalow               1652.0
                 Detached House         1913.0
                 Semi-detached House    1313.0
                                         ...  
BT9              Detached House         3397.0
                 Semi-detached House    3000.0
                 Terrace House          1251.0
                 Townhouse              2269.0
Belfast          Detached House            NaN
Name: Rates, Length: 78, dtype: float64

In [193]:
# Checking for null rates values
df.loc[df['Rates'].isna()]

,Bathrooms,Postcode,Bedrooms,Rates,Receptions,Price (£),Postcode_number,Address,Style,Estate Agent,Belfast District
275,3,Belfast,4,NaN,2,565000.0,Belfast,"The Pottinger, Baron Lane",Detached House,Colliers New Homes,None
357,2,Belfast,4,NaN,2,865000.0,BT7,"The Deane (site 18), The Hamptons",Detached House,GOC Estate Agents,None
358,2,Belfast,4,NaN,2,865000.0,BT7,"The Heath (site 32), The Hamptons",Detached House,GOC Estate Agents,None
359,2,Belfast,4,NaN,2,835000.0,BT7,"The Heath (site 10), The Hamptons",Detached House,GOC Estate Agents,None
360,2,Belfast,4,NaN,2,1100000.0,BT7,"The Satchwell (site 9), The Hamptons",Detached House,GOC Estate Agents,None
361,2,Belfast,4,NaN,2,1100000.0,BT7,"The Satchwell (site 8), The Hamptons",Detached House,GOC Estate Agents,None
362,3,Belfast,4,NaN,2,835000.0,BT7,"The Deane (site 3a), The Hamptons",Detached House,GOC Estate Agents,None
363,3,Belfast,4,NaN,2,835000.0,BT7,"The Deane (site 3), The Hamptons",Detached House,GOC Estate Agents,None
676,None,BT12 5GF,2,NaN,1,165000.0,BT12,Apartment 58 Victoria Place,Apartment,CPS,West


# It would appear as though we still have some null rates values that need addressed. Given that 8/9 of the properties are detached homes with 4 bedrooms and 2 receptions, we will use this information to get the average rates value of a reflective home and replace the nulls. I will then take the same approach with the remaining apartment.

In [194]:
rates_detached = df.loc[(df['Bedrooms'] == '4') & (df['Receptions'] == '2') & (df['Style'] == 'Detached House')]
rates_apartment = df.loc[(df['Bedrooms'] == '2') & (df['Receptions'] == '1') & (df['Style'] == 'Apartment')]
rates_mean_detached = rates_detached['Rates'].mean().round(2)
rates_mean_apartment = rates_apartment['Rates'].mean().round(2)

In [195]:
# Fill the NaNs with the new rates values
df.loc[df['Style'] == 'Detached House', 'Rates'] = df.loc[df['Style'] == 'Detached House', 'Rates'].fillna(rates_mean_detached)
df.loc[df['Style'] == 'Apartment', 'Rates'] = df.loc[df['Style'] == 'Apartment', 'Rates'].fillna(rates_mean_apartment)

# The final bit of data cleansing we need to do is addressing the null bathrooms, bedrooms and receptions. Having looked at the data, its safe to assume that every detached and semi-detached has at least 1 reception and 1 bathroom, for any property that has 2 receptions it is likely that they also have 2 bathrooms.

In [196]:
df.loc[((df['Receptions'] == '1') | (df['Receptions'].isna())) & (df['Bathrooms'].isna()), 'Bathrooms'] = '1'
df.loc[(df['Receptions'] == '2') & (df['Bathrooms'].isna()) & (df['Style'] != 'Apartment'), 'Bathrooms'] = '2'

# Assuming every house style has at least 1 reception and 1 bathroom
df.loc[df['Receptions'].isna(), 'Receptions'] = '1'
df.loc[df['Bathrooms'].isna(), 'Bathrooms'] = '1'

In [197]:
# Checking for null bedrooms
df.loc[df['Bedrooms'].isna()]

# Assuming the Terrace house and Semi-Detached home will have at least 2 bedrooms
df['Bedrooms'].fillna('2', inplace=True)

In [198]:
# Dealing with null belfast districts
df.loc[df['Belfast District'].isna()]

,Bathrooms,Postcode,Bedrooms,Rates,Receptions,Price (£),Postcode_number,Address,Style,Estate Agent,Belfast District
275,3,Belfast,4,2018.79,2,565000.0,Belfast,"The Pottinger, Baron Lane",Detached House,Colliers New Homes,None
357,2,Belfast,4,2018.79,2,865000.0,BT7,"The Deane (site 18), The Hamptons",Detached House,GOC Estate Agents,None
358,2,Belfast,4,2018.79,2,865000.0,BT7,"The Heath (site 32), The Hamptons",Detached House,GOC Estate Agents,None
359,2,Belfast,4,2018.79,2,835000.0,BT7,"The Heath (site 10), The Hamptons",Detached House,GOC Estate Agents,None
360,2,Belfast,4,2018.79,2,1100000.0,BT7,"The Satchwell (site 9), The Hamptons",Detached House,GOC Estate Agents,None
361,2,Belfast,4,2018.79,2,1100000.0,BT7,"The Satchwell (site 8), The Hamptons",Detached House,GOC Estate Agents,None
362,3,Belfast,4,2018.79,2,835000.0,BT7,"The Deane (site 3a), The Hamptons",Detached House,GOC Estate Agents,None
363,3,Belfast,4,2018.79,2,835000.0,BT7,"The Deane (site 3), The Hamptons",Detached House,GOC Estate Agents,None
463,2,Belfast,2,1087.00,1,230000.0,BT4,"Orchard Hall Apartment 4, Orchard Hall, Castle...",Apartment,Simon Brien Residential,None
464,1,Belfast,1,1087.00,1,250000.0,BT4,"Orchard Hall Apartment 9, Orchard Hall, Castle...",Apartment,Simon Brien Residential,None


In [199]:
areas_of_belfast = {
    'BT4': 'East',
    'BT7': 'South',
    'BT8': 'South',
    'BT9': 'South',
    'BT36': 'North'}

df['Belfast District'] = df['Belfast District'].fillna(df['Postcode_number'].map(areas_of_belfast))
df = df.dropna(subset=['Belfast District'])

In [200]:
# Although these postcode numbers cover two districts, for simplicity, I am going to change them.
df.loc[df['Postcode_number'] == 'BT17', 'Belfast District'] = 'West'
df.loc[df['Postcode_number'] == 'BT10', 'Belfast District'] = 'South-West'

In [201]:
# Final step is to convert the relevant columns into integers
df[['Bedrooms','Bathrooms','Receptions']] = df[['Bedrooms','Bathrooms','Receptions']].astype(int)

In [202]:
print(f"Number of null bathrooms: {len(df.loc[df['Bathrooms'].isna()])}")
print(f"Number of null bedrooms: {len(df.loc[df['Bedrooms'].isna()])}")
print(f"Number of null receptions: {len(df.loc[df['Receptions'].isna()])}")
print(f"Number of null rates: {len(df.loc[df['Rates'].isna()])}")
print(f"Number of null Belfast Districts: {len(df.loc[df['Belfast District'].isna()])}")

Number of null bathrooms: 0
Number of null bedrooms: 0
Number of null receptions: 0
Number of null rates: 0
Number of null Belfast Districts: 0


In [203]:
df.to_csv('clean_df.csv')
df

,Bathrooms,Postcode,Bedrooms,Rates,Receptions,Price (£),Postcode_number,Address,Style,Estate Agent,Belfast District
0,2,BT8 8FX,4,1652.05,2,335000.0,BT8,5 Haddo Crescent,Townhouse,Ulster Property Sales,South
1,2,BT8 8GU,3,1129.41,1,229950.0,BT8,84 The Demesne,Townhouse,Ulster Property Sales,South
2,2,BT8 8FX,4,1652.05,2,335000.0,BT8,5 Haddo Crescent,Townhouse,Ulster Property Sales,South
3,1,BT8 7YG,2,1045.75,1,169000.0,BT8,17 Greer Park Heights,Townhouse,Fetherstons,South
4,2,BT8 6BA,4,1882.35,1,245000.0,BT8,5 Croft Hill Cottages,Townhouse,Ulster Property Sales,South
...,...,...,...,...,...,...,...,...,...,...,...
772,1,BT36 7EY,2,586.91,2,79950.0,BT36,17 Voltaire Gardens,Semi-detached House,Pinpoint Property,North
773,1,BT14 6LF,4,630.39,2,119950.0,BT14,19 Clifton Crescent,Townhouse,House To Home Estate Agents,North-West
774,1,BT11 9QW,2,652.13,1,129950.0,BT11,10 Gweedore Crescent,Townhouse,Ulster Property Sales,West
775,3,BT17 0JE,5,3043.25,3,399950.0,BT17,11 Glengoland Gardens,Bungalow,Ulster Property Sales,West


# We now have a clean dataframe that is ready for analysis.